In [8]:
import mysql.connector

In [9]:
conn = mysql.connector.connect(
    user='root',
    password='Qadeerh03',
    host='127.0.0.1',
    database='patient',
    port=3306
)

In [10]:
# Create a cursor object
cursor = conn.cursor()

In [11]:
# Query to find all tables in the database
query = "SELECT * FROM patient_data"

In [12]:
cursor.execute(query)
results = cursor.fetchall()

In [13]:
for row in results:
    print(row)

(1, 'Joe', datetime.date(2024, 11, 28), '+353', 'fdeafvesv@gmail.com', 'sq', 'fwAE', 'gvsgvr', 'gsegvesa')
